In [16]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

%matplotlib inline

# Create the example time series data
To avoid repeating some of the contents that have been covered by others, I will only include a link here. This [post](https://towardsdatascience.com/basic-time-series-manipulation-with-pandas-4432afee64ea) presents some nice basic stuff, such as creating time seires.

In [68]:
date_rng = pd.date_range(start='2020-01-01', end='2020-12-31', freq='H')
df = pd.DataFrame(date_rng, columns=['StartTime'])
df['Data'] = np.random.randint(0,100,size=(len(date_rng)))

In [69]:
df.head()

,StartTime,Data
0,2020-01-01 00:00:00,1
1,2020-01-01 01:00:00,64
2,2020-01-01 02:00:00,83
3,2020-01-01 03:00:00,0
4,2020-01-01 04:00:00,46


# Make time seires complete

In [71]:
# Let's first delibrately remove some time intervals
# Remove the time intervals of 7 , 8 and 9 am

mask = df.StartTime.dt.hour.isin([7, 8, 9])
df = df[~mask]

In [72]:
# Then add these interval placeholders by using resample

df = df.set_index('StartTime').resample('1H').sum()
df.reset_index(level=0, inplace=True)

In [74]:
df.head(10)

,StartTime,Data
0,2020-01-01 00:00:00,1
1,2020-01-01 01:00:00,64
2,2020-01-01 02:00:00,83
3,2020-01-01 03:00:00,0
4,2020-01-01 04:00:00,46
5,2020-01-01 05:00:00,5
6,2020-01-01 06:00:00,72
7,2020-01-01 07:00:00,0
8,2020-01-01 08:00:00,0
9,2020-01-01 09:00:00,0


# Perform calculations with rolling time windows
The example below shows that, for any given time interval, how we can calculate the mean based on the values from the same time interval from the past 5 weekdays

In [75]:
# First create some additional columns: IsWeekday and Starthour
mask_weekends = df.StartTime.dt.weekday.isin([5, 6])
df['IsWeekday'] = 1
df.loc[mask_weekends, 'IsWeekday'] = 0
df['StartHour'] = df.StartTime.dt.hour

In [76]:
df.head()

,StartTime,Data,IsWeekday,StartHour
0,2020-01-01 00:00:00,1,1,0
1,2020-01-01 01:00:00,64,1,1
2,2020-01-01 02:00:00,83,1,2
3,2020-01-01 03:00:00,0,1,3
4,2020-01-01 04:00:00,46,1,4


In [77]:
# Then we could apply the 'rolling' function. Notice that "closed='left'" means that the value from current interval will not be used.
res_series = df.groupby(by=['IsWeekday', 'StartHour'])['Data'].rolling(window=5, closed="left").mean()

In [78]:
# The result is a series with multiple indices
res_series

IsWeekday  StartHour      
0          0          72       NaN
                      96       NaN
                      240      NaN
                      264      NaN
                      408      NaN
                              ... 
1          23         8615    54.4
                      8639    57.2
                      8711    63.6
                      8735    73.2
                      8759    70.8
Name: Data, Length: 8761, dtype: float64

In [79]:
# Convert this series to a df and merge it with the original one. Done!
res_df = res_series.to_frame()
res_df.index=res_df.index.get_level_values(2)
res_df.rename(columns={'Data': 'Mean_5D'}, inplace=True) 
df = df.merge(res_df, left_index=True, right_index=True, how='left')

In [66]:
df.tail(20)

,StartTime,Data,IsWeekday,StartHour,Mean_5D
8741,2020-12-30 05:00:00,20,1,5,72.0
8742,2020-12-30 06:00:00,85,1,6,44.6
8743,2020-12-30 07:00:00,0,1,7,0.0
8744,2020-12-30 08:00:00,0,1,8,0.0
8745,2020-12-30 09:00:00,0,1,9,0.0
8746,2020-12-30 10:00:00,11,1,10,42.8
8747,2020-12-30 11:00:00,71,1,11,48.4
8748,2020-12-30 12:00:00,48,1,12,55.2
8749,2020-12-30 13:00:00,50,1,13,29.0
8750,2020-12-30 14:00:00,80,1,14,61.4
